In [1]:
import numpy as np
import time
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

import tensorflow as tf
from Dataset import BitcoinData
from TradingEnvironment import TradingEnvironment
from Models import DQN

tf.keras.utils.disable_interactive_logging()

NUM_EPISODES = 300

In [5]:
bitcoin_data = BitcoinData()

def cryptocurrencies_trading(load_model_value = -1):
    environment = TradingEnvironment(bitcoin_data)

    observation_space = environment.get_observation_space()
    action_space = environment.get_action_space()
    episode = 0

    agent = DQN(observation_space, action_space, load_model_value=load_model_value)

    while episode < NUM_EPISODES:
        episode += 1
        state = environment.reset()
        step = 0
        while True:
            step += 1
            action = agent.act(np.reshape(state, [1, observation_space]))
            next_state, reward, done, info = environment.step(action - 1)

            agent.memory.remember(state, action, reward, next_state, done)
            agent.learn()

            state = next_state

            if done:
                print("Episode: {}/{}, profit: {:.4f}, min_profit: {:.4}, max_profit: {:.4}".format(
                    episode, NUM_EPISODES, environment.percent_profit, environment.min_percent_profit, environment.max_percent_profit))
                break

            if (step - 1) % 100 == 0:
                print("Step: {}, profit: {:.4f}, min_profit: {:.4f}, max_profit: {:.4f}".format(
                    step - 1, environment.percent_profit, environment.min_percent_profit, environment.max_percent_profit))
                
        agent.model.save(f"Models/Checkpoint/Model_{episode}.keras")

Dataset already downloaded!
Reading dataset...
Dataset read!


In [4]:
cryptocurrencies_trading()

TypeError: cryptocurrencies_trading() takes 0 positional arguments but 1 was given